# **Alter_System_Prompts_in_LLMs**

**Note - This script executed in Google Colab**

- Here we alter the **system default prompt** and create the **new system prompt**
- https://replicate.com/meta/llama-2-7b-chat

- **We have 2 type Prompt**
  - **Instruction Token(Instruction Prompt/Input promt/Q we ask)**
  - **System Token( System default prompt, prompt which is already there in llm backend)**

- System prompt always **default inside LLM**, now we can updated this system prompt like below
- Inside Template we pass system Prompt +Instruction prompt

- Different LLM will have different Token, we need to check the LLM's documentation. For Llama2 -  **"[INST]", "[/INST]"** are for Instruction prompt Token and "<<SYS>>\n", "\n<<SYS>>\n\n"  are for System prompt Token

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 937.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

### **Connect to Hugging face account**
-  This to connect to **hugging face account**. This is another way of login from notebook_login() from hugging face  not from key we are defining in colab  or  or cli login

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## **1. Import all Libraries**

In [6]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## **2. Define Llama2 Model**

-  Create **LLM wrapper**
- Use this Open Source **Meta Llama2** model via **Hugging face** and pass **Q** to this Llam2 Model

In [7]:
model="meta-llama/Llama-2-7b-chat-hf"

###  **Invoke Llam2 Model's Tokenizer**

- If we are using **hugging face pipeline** Not doing any **Vector DB** Creation, then better to use **specific model's tokenizer** only - **AutoTokenizer.from_pretrained** (Ex this problem),
- Directly doing langchain framework(Not hugging face pipeline) and it involves to write embedding in **vector DB**, That time we can use **Openai embedding model or hugging faces sentence-transformers**

In [8]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### **Define Hugging face pipeline parameter**
- Here it downloads model

In [9]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

### **Invoke Llama2 model via Hugging face pipeline**
- Execute LLM Wrapper

In [11]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
#Just to test, Execute Llama2 as general llm model - it generates from its base knowledge
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Please provide a concise summary of the Book Harry Potter and the Philosopher\'s Stone.\nHarry Potter and the Philosopher\'s Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter and his friends Ron Weasley and Hermione Granger as they attend Hogwarts School of Witchcraft and Wizardry.\nHarry discovers that he is famous in the wizarding world, as his parents were killed by the dark wizard Lord Voldemort. He also learns that he is the chosen one, destined to defeat Voldemort and save the wizarding world.\nThe trio embarks on a quest to find and protect the Philosopher\'s Stone, a powerful magical object that can grant eternal life. Along the way, they encounter various obstacles, including the Potions master Severus Snape, the Hogwarts ghost Nearly Headless Nick, and the evil wizard Quirrell.\nThe novel is the first in the Harry Potter series and introduces many of the recurring themes, characters, and magical elements t

## **3. Create Custome Prompt**

- Here we alter the **system default prompt** and create the **new system prompt**

- **We have 2 type Prompt**
  - **Instruction Token(Instruction Prompt/Input promt/Q we ask)**
  - **System Token( System default prompt, prompt which is already there in llm backend)**

- System prompt always **default inside LLM**, now we can updated this system prompt like below
- Inside Template we pass system Prompt +Instruction prompt

- Different LLM will have different Token, we need to check the LLM's documentation. For Llama2 -  **"[INST]", "[/INST]"** are for Instruction prompt Token and "<<SYS>>\n", "\n<<SYS>>\n\n"  are for System prompt Token

### **Text Translation Custom Prompt Using Default System Prompt**

In [13]:
## Lllam2 Models Instruction prompt Tokens
B_INST, E_INST= "[INST]", "[/INST]"  # Begin of instruction Token , End of instruction Token

## Lllam2 Models system prompt Tokens
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"  # Begin of System Token , End of System Token

In [14]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [ ]:
#instruction = "Who won the T20 Cricket World Cup 2022"

In [15]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [17]:
instruction = "Convert the following text from English to French: \n\n{text}"

#### **Combine Custom systemTemplate + Instruction Template**

In [18]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [19]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [26]:
text ="How are you"

LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [27]:
LLM_Chain.run(text)

'[INST]<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\nConvert the following text from English to French: \n\nHow are you[/INST]  Sure! Here\'s the translation of "How are you" from English to French:\n\nComment vas-tu?'

### **Text Translation Custum Prompt Using Updated/Custom System Prompt**

In [21]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\n'

In [22]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [23]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [24]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [25]:
text ="How are you"

LLM_Chain=LLMChain(llm=llm, prompt=prompt)
print(LLM_Chain.run(text))

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

How are you[/INST]  Sure! Here's the translation of "How are you" from English to French:

Comment vas-tu?


# **END**